# Accessibility report of campaign website's
## The result is a csv with the candidate name and unique identifier (candidate id) along with the fields from their campaign website's axe report on accessibility.

In [136]:
import json
import pandas as pd
import csv
from selenium import webdriver
from axe_selenium_python import Axe
import requests
from requests.exceptions import ConnectionError
import os

In [2]:
candidate_data = pd.read_csv(r'localcandidate-sample.csv')

### The following are all the available features in this dataset:

In [5]:
list(candidate_data.columns)

['Election year',
 'Stage ID',
 'Stage',
 'Election district',
 'Election District type',
 'State',
 'Race type',
 'Race ID',
 'Office level',
 'Office branch',
 'Office name',
 'Office ID',
 'District name',
 'District type',
 'District OCDID',
 'Seats up for election',
 'Race URL',
 'Name',
 'First name',
 'Last name',
 'Ballotpedia URL',
 'Candidate ID',
 'Person ID',
 'Gender',
 'Party affiliation',
 'Incumbent?',
 'Election date',
 'Stage canceled?',
 'Candidate status',
 'Stage is ranked choice?',
 'Ranked-choice voting round',
 'Partisan primary?',
 'Campaign email',
 'Other email',
 'Campaign website',
 'Personal website',
 'Campaign Facebook',
 'Personal Facebook',
 'Campaign Twitter',
 'Personal Twitter',
 'Campaign Instagram',
 'Personal Instagram',
 'Campaign mailing address',
 'LinkedIn',
 'image']

### The following features will be included in the final dataset:
- Candidate ID
- Person ID
- First Name
- Last Name
- Campaign Website

### The following features will be included from the axe report:
- Violation #
- Description
- Help
- HelpUrl
- Id
- Impact
- Tags


### Features Ignored:
- Nodes

This function accepts a url and creates an axe report in json format. Then parses the json file to create a dictionary with all the elements nested.

In [133]:
def test_site(url: str) -> dict:
    # Add the path the geckodriver you downloaded earlier
    # The following is an example
    driver = webdriver.Firefox()
    driver.get(url)
    axe = Axe(driver)
    # Inject axe-core javascript into page.
    axe.inject()
    # Run axe accessibility checks.
    results = axe.run()
    # Write results to file
    axe.write_results(results, 'access.json')
    driver.close()
    # Assert no violations are found
    # assert len(results["violations"]) == 0, axe.report(results["violations"])
    with open('access.json', 'r') as content:
        data = json.load(content)
    
    return data

In [ ]:
# Testing the function:

test_data = test_site("http://natalieisawesome.com/")

Converting the dataframe into the result dataframe by including the violation details

In [134]:
result = pd.DataFrame(columns = ["Candidate ID", "Person ID", "First Name", "Last Name", "Campaign Website",
                                 "Violation #", "Description", "Help", "HelpUrl", "Id", "Impact", "Tags"])

In [139]:
for index, row in candidate_data.iterrows():
    website = row['Campaign website']
    # checking if there is a value for 'campaign website'
    if not pd.isna(website):
        try:
            # checking if this website is valid
            request = requests.get(website)
        except ConnectionError:
            print(row['First name'] + " " + row['Last name'] + '\'s campaign website does not exist')
            continue
        
        # calling the function to create json report file
        axe_report = test_site(website)
        axe_violations = axe_report['violations']
        count = len(axe_violations)
        print("Checking violations in " + row['First name'] + " " + row['Last name'] + '\'s campaign website')
        
        # appending the data frame
        # each violation is added as a new observation and tracked with vilation #
        for i in range(count):
            violation_num = i + 1
            violation_desc = axe_violations[i]['description']
            violation_help = axe_violations[i]['help']
            violation_helpurl = axe_violations[i]['helpUrl']
            violation_id = axe_violations[i]['id']
            violation_impact = axe_violations[i]['impact']
            violation_tags = axe_violations[i]['tags']
            result = result.append({'Candidate ID': row['Candidate ID'],'Person ID':row['Person ID'], 
                                    'First Name': row['First name'], 'Last Name': row['Last name'], 
                                    'Campaign Website': row['Campaign website'], 'Violation #': violation_num,
                                    'Description': violation_desc, 'Help': violation_help, 'HelpUrl': violation_helpurl,
                                    'Id': violation_id, 'Impact': violation_impact, 'Tags': violation_tags}, ignore_index=True)
    # remove the json file once done
    if os.path.exists('access.json'):
        os.remove('access.json')
         

Checking violations in Natalie Fleming's campaign website
Checking violations in Craig Williams's campaign website
Checking violations in Weston Ulbrich's campaign website
Checking violations in Jeff Howe's campaign website
Checking violations in Bob Carney's campaign website
Checking violations in Matthew McIntyre's campaign website
Checking violations in Daniel Rosenthal's campaign website
Checking violations in Tom Young's campaign website
Checking violations in Leon Benjamin's campaign website
Checking violations in Todd Hunter's campaign website
Checking violations in William Driscoll's campaign website
Checking violations in Freddy Ramirez's campaign website
Checking violations in Tom Dent's campaign website
Checking violations in Padraic Rafferty's campaign website
Checking violations in Michele Meyer's campaign website
Checking violations in Mark Gamba's campaign website
Checking violations in Lynne Walz's campaign website
Checking violations in Tim Rudd's campaign website
Jon 

In [138]:
result.head()

,Candidate ID,Person ID,First Name,Last Name,Campaign Website,Violation #,Description,Help,HelpUrl,Id,Impact,Tags
0,65809,304885,Natalie,Fleming,http://natalieisawesome.com/,1,Ensures role attribute has an appropriate valu...,ARIA role must be appropriate for the element,https://dequeuniversity.com/rules/axe/3.1/aria...,aria-allowed-role,minor,"[cat.aria, best-practice]"
1,65809,304885,Natalie,Fleming,http://natalieisawesome.com/,2,Ensures all page content is contained by landm...,All page content must be contained by landmarks,https://dequeuniversity.com/rules/axe/3.1/regi...,region,moderate,"[cat.keyboard, best-practice]"
2,79281,326195,Craig,Williams,http://williamsforiowa.com/,1,Ensures role attribute has an appropriate valu...,ARIA role must be appropriate for the element,https://dequeuniversity.com/rules/axe/3.1/aria...,aria-allowed-role,minor,"[cat.aria, best-practice]"
3,79281,326195,Craig,Williams,http://williamsforiowa.com/,2,Ensures the order of headings is semantically ...,Heading levels should only increase by one,https://dequeuniversity.com/rules/axe/3.1/head...,heading-order,moderate,"[cat.semantics, best-practice]"
4,79281,326195,Craig,Williams,http://williamsforiowa.com/,3,Ensures the page has only one main landmark an...,Page must have one main landmark,https://dequeuniversity.com/rules/axe/3.1/land...,landmark-one-main,moderate,"[cat.semantics, best-practice]"


Converting the dataframe to CSV:

In [132]:
result.to_csv('localcandidate_website_violations.csv', encoding='utf-8')